# 数据扩充算法   
### 2020年2月7日 WenQLi  
## 一、Fancy PCA 算法

### 1、PCA算法简介（降维用）

   PCA（Principal Components Analysis）即主成分分析，是图像处理中经常用到的降维方法，在处理有关数字图像处理方面的问题时，比如经常用的图像的查询问题，在一个几万或者几百万甚至更大的数据库中查询一幅相近的图像。
    通常的方法是对图像库中的图片提取响应的特征，如颜色，纹理，sift，surf，vlad等特征，然后将其保存，建立响应的数据索引，然后对要查询的图像提取相应的特征，与数据库中的图像特征对比，找出与之最近的图片。
    这里，如果我们为了提高查询的准确率，通常会提取一些较为复杂的特征，如sift，surf等，一幅图像有很多个这种特征点，每个特征点又有一个相应的描述该特征点的128维的向量，设想如果一幅图像有300个这种特征点，那么该幅图像就有300*vector（128维）个，如果我们数据库中有一百万张图片，这个存储量是相当大的，建立索引也很耗时，如果我们对每个向量进行PCA处理，将其降维为64维，可以有效的减少检索时间。

## 二、算法详解

### 1、计算数据集的均值、方差等统计信息；
### 2、根据公式，计算协方差矩阵；
（还有一种方法，先让样本矩阵中心化，即每一维度减去该维度的均值，然后直接用新的到的样本矩阵乘上它的转置，然后除以(N-1)即可）
### 3、计算协方差矩阵的特征向量和特征值；
### 4、选择分组成为特征矢量：
求出协方差矩阵的特征值及特征向量之后，按照特征值由大到小进行排列，这将给出成分的重要性级别。可以忽略那些重要性很小的成分，当然这会丢失一些信息，但是如果对应的特征值很小，不会丢失很多信息。如果已经忽略了一些成分，那么最后的数据集将有更少的维数，精确地说，如果原始数据是n维的，选择了前p个主要成分，那么现在的数据将仅有p维。现在我们要做的是组成一个模式矢量，这只是几个矢量组成的矩阵的一个有意思的名字而已，它由你保持的所有特征矢量构成，每一个特征矢量是这个矩阵的一列。

### 5、得到降维后的数据

<img src="./gs_1.jpg", width=640, heigth=61>

In [1]:
%%html
<img src="./gs_1.jpg", width=640, heigth=61>

其中rowFeatureVector是由模式矢量作为列组成的矩阵的转置，因此它的行就是原来的模式矢量，而且对应最大特征值的特征矢量在该矩阵的最上一行。rowdataAdjust是每一维数据减去均值后，所组成矩阵的转置，即数据项目在每一列中，每一行是一维，对我们的样本来说即是，第一行为x维上数据，第二行为y维上的数据。FinalData是最后得到的数据，数据项目在它的列中，维数沿着行。

### 6、Fancy PCA算法
通常图像具有RGB三个通道，例如一张256 * 256的彩色图像具有(256, 256, 3)的尺寸，每一个通道形成一组数据，将图像尺寸改成((256*256), 3)，进而将图像转换成一个三维向量。然后对其执行PCA，得到一个3 * 3的协方差矩阵，通过该矩阵求得特征值与特征向量，特征值与特征向量按从大到小的顺序进行排序。计算公式如下：

In [2]:
%%html
<img src="./gs_2.png", width=300, heigth=61>

其中[P1,P2,P3]表示特征向量，[λ1,λ2,λ3]表示特征值。在论文中α是一个服从均值为0，标准差为0.1的高斯分布。对于同一张图像，α = α1 = α2 = α3。上述公式得到的结果是一个尺寸为(3, 1)的向量，原图像的每个像素(R, G, B)都加上这个向量得到新的目标图像。

In [3]:
import numpy as np


def fancy_pca(img, alpha_std=0.1):
    '''
    INPUTS:
    img:  numpy array with (h, w, rgb) shape, as ints between 0-255)
    alpha_std:  how much to perturb/scale the eigen vecs and vals
                the paper used std=0.1
    RETURNS:
    numpy image-like array as float range(0, 1)
    NOTE: Depending on what is originating the image data and what is receiving
    the image data returning the values in the expected form is very important
    in having this work correctly. If you receive the image values as UINT 0-255
    then it's probably best to return in the same format. (As this
    implementation does). If the image comes in as float values ranging from
    0.0 to 1.0 then this function should be modified to return the same.
    Otherwise this can lead to very frustrating and difficult to troubleshoot
    problems in the image processing pipeline.
    This is 'Fancy PCA' from:
    # http://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf
    #######################
    #### FROM THE PAPER ###
    #######################
    "The second form of data augmentation consists of altering the intensities
    of the RGB channels in training images. Specifically, we perform PCA on the
    set of RGB pixel values throughout the ImageNet training set. To each
    training image, we add multiples of the found principal components, with
    magnitudes proportional to the corresponding eigenvalues times a random
    variable drawn from a Gaussian with mean zero and standard deviation 0.1.
    Therefore to each RGB image pixel Ixy = [I_R_xy, I_G_xy, I_B_xy].T
    we add the following quantity:
    [p1, p2, p3][α1λ1, α2λ2, α3λ3].T
    Where pi and λi are ith eigenvector and eigenvalue of the 3 × 3 covariance
    matrix of RGB pixel values, respectively, and αi is the aforementioned
    random variable. Each αi is drawn only once for all the pixels of a
    particular training image until that image is used for training again, at
    which point it is re-drawn. This scheme approximately captures an important
    property of natural images, namely, that object identity is invariant to
    change."
    ### END ###############
    Other useful resources for getting this working:
    # https://groups.google.com/forum/#!topic/lasagne-users/meCDNeA9Ud4
    # https://gist.github.com/akemisetti/ecf156af292cd2a0e4eb330757f415d2
    '''

    orig_img = img.astype(float).copy()

    img = img / 255.0  # rescale to 0 to 1 range

    # flatten image to columns of RGB
    img_rs = img.reshape(-1, 3)
    # img_rs shape (640000, 3)

    # center mean
    img_centered = img_rs - np.mean(img_rs, axis=0)

    # paper says 3x3 covariance matrix
    img_cov = np.cov(img_centered, rowvar=False)

    # eigen values and eigen vectors
    eig_vals, eig_vecs = np.linalg.eigh(img_cov)

#     eig_vals [0.00154689 0.00448816 0.18438678]

#     eig_vecs [[ 0.35799106 -0.74045435 -0.56883192]
#      [-0.81323938  0.05207541 -0.57959456]
#      [ 0.45878547  0.67008619 -0.58352411]]

    # sort values and vector
    sort_perm = eig_vals[::-1].argsort()
    eig_vals[::-1].sort()
    eig_vecs = eig_vecs[:, sort_perm]

    # get [p1, p2, p3]
    m1 = np.column_stack((eig_vecs))

    # get 3x1 matrix of eigen values multiplied by random variable draw from normal
    # distribution with mean of 0 and standard deviation of 0.1
    m2 = np.zeros((3, 1))
    # according to the paper alpha should only be draw once per augmentation (not once per channel)
    alpha = np.random.normal(0, alpha_std)

    # broad cast to speed things up
    m2[:, 0] = alpha * eig_vals[:]

    # this is the vector that we're going to add to each pixel in a moment
    add_vect = np.matrix(m1) * np.matrix(m2)

    for idx in range(3):   # RGB
        orig_img[..., idx] += add_vect[idx]

    # for image processing it was found that working with float 0.0 to 1.0
    # was easier than integers between 0-255
    # orig_img /= 255.0
    orig_img = np.clip(orig_img, 0.0, 255.0)

    # orig_img *= 255
    orig_img = orig_img.astype(np.uint8)

    # about 100x faster after vectorizing the numpy, it will be even faster later
    # since currently it's working on full size images and not small, square
    # images that will be fed in later as part of the post processing before being
    # sent into the model
#     print("elapsed time: {:2.2f}".format(time.time() - start_time), "\n")

    return orig_im

## 参考文献：
https://blog.csdn.net/b1055077005/article/details/96013305  
https://blog.csdn.net/a8039974/article/details/81285238  
https://github.com/pixelatedbrian/fortnight-furniture/blob/master/src/fancy_pca.py  